In [1]:
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

In [2]:
import spacy
import re
from spacy import displacy
import pandas as pd
import numpy as np

from adept.components.registry import ComponentsRegistry
from adept.components.sentencizer import Sentencizer
from adept.components.numeric import (NumericDimension, NumericExpand, NumericFraction, NumericMeasurement, NumericRange)
from adept.components.anatomical import AnatomicalEntity
from adept.components.traits import (CustomTraitsEntity, DiscreteTraitsEntity, NumericTraitsEntity)

from adept.postprocess.postprocessors.discrete import DiscreteTraitsPostprocessor
from adept.postprocess.postprocessors.custom import CustomTraitsPostprocessor
from adept.postprocess.postprocessors.measurement import MeasurementTraitsPostprocessor
from adept.scripts.helpers import get_descriptions

from adept.config import CORPUS_DIR, RAW_DATA_DIR

%env TOKENIZERS_PARALLELISM=(true | false)

env: TOKENIZERS_PARALLELISM=(true | false)


In [3]:
nlp = spacy.load("en_core_web_trf")

In [42]:
registry = ComponentsRegistry(nlp)
registry.add_components([
    Sentencizer,
    AnatomicalEntity,
    CustomTraitsEntity,
    DiscreteTraitsEntity,
    NumericTraitsEntity,
    NumericExpand,
    NumericDimension,
    NumericMeasurement,
    NumericRange,
    NumericFraction,
])

INIT custom_sentencizer
INIT custom_traits_entity
INIT numeric_trait_entity
INIT numeric_expand
INIT numeric_dimensions
INIT numeric_measurements
INIT numeric_range
INIT numeric_fraction


In [43]:
text = "Cherry plum, myrobalan Trees, sometimes suckering, 40-80 dm, not or slightly thorny. Twigs with axillary end buds, glabrous. Leaves deciduous; petiole 5-20 mm, glabrous except for a few hairs on adaxial surface, eglandular; blade ovate, elliptic, or obovate, 3-7 x 1.5-3.5 cm, base obtuse, margins singly to doubly crenate-serrate, teeth blunt, glandular, apex obtuse to acute, abaxial surface hairy along midribs and veins, adaxial glabrous. Inflorescences usually solitary flowers, sometimes 2-flowered fascicles. Pedicels (4-)10-18 mm, glabrous. Flowers blooming before leaf emergence; hypanthium campanulate, 2-4 mm, glabrous externally; sepals reflexed to spreading, oblong-ovate, 2-4 mm, margins glandular-toothed to nearly entire, <eciliate>, abaxial surface glabrous, adaxial hairy at bases; petals white (reddish pink in cultivars), elliptic to suborbiculate, 7-14 mm; ovaries glabrous. Drupes purple-red to yellow, <sometimes glaucous>, ovoid, ellipsoid, or globose, 15-30 mm, glabrous; mesocarps fleshy; stones ellipsoid to ovoid, +- to strongly flattened. 2n = 16."

In [44]:
doc = nlp(text)

/Users/ben/Library/Caches/pypoetry/virtualenvs/adept-igwYr31N-py3.9/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


mm
cm
mm
mm
mm
mm
mm
ERROR
2-


In [12]:
displacy.render(doc, style='ent', jupyter=True)

In [46]:
accdb = ACCDBTraits()



In [108]:
xslx = RAW_DATA_DIR / 'functional-trait-list.xlsx'
df = pd.read_excel(xslx, sheet_name='Pteridophyte traits')
# columns = list(map(str.lower, df.columns))

In [112]:
columns = list(map(str.lower, df.columns))

rename_cols = {
    'maximum plant height (m)': 'plant height (m)',
    'leaf minimum width cm': 'leaf min. width [cm]',
    'leaf maximum width cm': 'leaf max. width [cm]',
    'leaf minimum length cm': 'leaf min. length [cm]',
    'leaf maximum length cm': 'leaf max. length [cm]'
}

columns = [rename_cols.get(col, None) or col for col in columns]

for column in columns[:]:

    # Search for (xx) or [x)     
    match = re.search(r'(.*)[\(\[]([a-zμ]{1,2})[\)\]]', column)
    if match and not re.search('(min|max)', column):
        
        idx = columns.index(column)
        
        trait = match.group(1).split()
        unit = match.group(2)        
        new_cols = [f'{trait[0]} {mm}. {trait[1]} [{unit}]' for mm in ['min', 'max']]

        columns[idx:idx+1] = new_cols

print(columns)

['clonality', 'habit', 'habitat', 'plant height (m)', 'reproduction system', 'ploidy level (2n)', 'fertile fronds', 'leaf function', 'leaf architecture', 'leaf margin', 'no. vascular strands', 'venation', 'leaf min. width [cm]', 'leaf max. width [cm]', 'leaf min. length [cm]', 'leaf max. length [cm]', 'spinescence', 'stem pubescence', 'leaf pubescence', 'indumentum', 'lamina thickness', 'scales', 'no. sori per leaf', 'indusium', 'no. sporangia per sorus', 'spore shape', 'spore diameter (μm)', 'spore surface']
['clonality', 'habit', 'habitat', 'plant min. height [m]', 'plant max. height [m]', 'reproduction system', 'ploidy level (2n)', 'fertile fronds', 'leaf function', 'leaf architecture', 'leaf margin', 'no. vascular strands', 'venation', 'leaf min. width [cm]', 'leaf max. width [cm]', 'leaf min. length [cm]', 'leaf max. length [cm]', 'spinescence', 'stem pubescence', 'leaf pubescence', 'indumentum', 'lamina thickness', 'scales', 'no. sori per leaf', 'indusium', 'no. sporangia per sor

In [173]:
# from adept.traits.accdb import ACCDBTraits

# accdb = ACCDBTraits()

from collections import ChainMap
import itertools
from spacy.util import filter_spans
from spacy.tokens import Span

input_path = CORPUS_DIR / 'preprocessed-descriptions.csv'

class Postprocess():  
    
    xslx = RAW_DATA_DIR / 'functional-trait-list.xlsx'
    
    def __init__(self, taxon_group):
        
        self.processors = [
            DiscreteTraitsPostprocessor(taxon_group),
            MeasurementTraitsPostprocessor(taxon_group),
            CustomTraitsPostprocessor(taxon_group)
        ] 
        
        self._columns = self.get_columns(taxon_group)
        
        self._data = []
        
    def get_columns(self, taxon_group):
        sheet_name = f'{taxon_group.capitalize()} traits'
        df = pd.read_excel(self.xslx, sheet_name=sheet_name)
        columns = list(map(str.lower, df.columns))
        columns += ['root min. depth [cm]', 'root max. depth [cm]']
        # Replace plant height with min/ax
        idx = columns.index('plant height [m]')
        columns[idx:idx+1] = ['plant min. height [m]', 'plant max. height [m]']
        return columns
        
        
    def process_doc(self, doc):
        data = {}
        ents = doc.ents
        for processor in self.processors:
            traits = processor(doc)
            data.update(ChainMap(*[trait.value for trait in traits if trait]))            
            new_ents = [Span(doc, ent.start, ent.end, label=label) for label, ent in processor.get_ents_log()]
            ents = itertools.chain(new_ents, ents)

        doc.ents = filter_spans(ents)    
        return data
    
    def __call__(self, doc, taxon, source):   

        data = self.process_doc(doc)

        # AARGGGH The database does not have the same name as the output file         
        name_aliases = [
            ('life cycle', 'life form'),
            ('flower structure', 'flower architecture')
        ]
        
        for old_name, new_name in name_aliases:        
            data[new_name] = data.pop(old_name, None)

        missing_columns = [c for c in data.keys() if c not in self._columns]
        
        if missing_columns:
            for col in missing_columns:
                print(f'MISSING COL {col}:', data[col])
            
            raise Exception('Columns missing', missing_columns)
       
        data['taxon'] = taxon
        data['source'] = source   

        return data
        
    def todf(self):
        columns = ['taxon', 'source'] + self._columns
        return pd.DataFrame(self._data, columns=columns)
        
        
postprocessors = {}
        
for i, description in enumerate(get_descriptions(input_path)):


    doc = nlp('puberulent ' + description.text)
    taxon_group = description.taxon_group.lower()
    
    try:
        postprocessors[taxon_group]
    except KeyError:
        postprocessors[taxon_group] = Postprocess(taxon_group)

    data = postprocessors[taxon_group](doc, description.taxon, description.source)
    
    # print(description.text)
    
    standard_ents = ['QUANTITY', 'TRAIT', 'PART', 'CARDINAL']
    labels = {ent.label_ for ent in doc.ents}
    colour = lambda l: '#DDDDDD' if l in standard_ents else '#9AD943'
    
    colours = {label: colour(label) for label in labels}
    
    colours['PART'] = '#FFEB5D'
    options = {
        'colors': colours
    }
    
    displacy.render(doc, style='ent', jupyter=True, options=options)
    print(data)
    
    break
    
    print(i)
    
    if i > 2:
        break

    



/Users/ben/Library/Caches/pypoetry/virtualenvs/adept-igwYr31N-py3.9/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


ERROR
3-


{'dispersion axillary': {'seed winged'}, 'dispersal mode': {'anemochory'}, 'flower colour': {'black', 'pink', 'red', 'white', 'brown'}, 'inflorescence arrangement': {'capitulum', 'corymb'}, 'leaf architecture': {'sessile'}, 'leaf apex': {'subulate'}, 'leaf arrangement': {'pinnate'}, 'leaf shape': {'lanceolate', 'setiform', 'linear'}, 'perennial organ': {'stem', 'stolon'}, 'clonality': {'stoloniferous'}, 'habit': {'erect leafy', 'scrambler', 'herb'}, 'indumentum': {'pubescent', 'hairs absent', 'glabrous'}, 'seed min. width [mm]': 1.8, 'seed max. width [mm]': 2.0, 'leaf min. width [cm]': 20.0, 'leaf max. width [cm]': 20.0, 'plant min. height [m]': 0.08, 'plant max. height [m]': 0.6, 'life form': {'perennial'}, 'flower architecture': {'naked'}, 'taxon': 'Achillea millefolium', 'source': 'ecoflora'}


In [30]:
# print('HIIII')


def aggregate(cell):
    
    _ensure_set = lambda x: x if isinstance(x, set) else set([x])

    if cell.dtype in ['int64', 'float64']:
        return round(cell.mean(), 2)
    else:
        return set.union(*[_ensure_set(c) for c in cell if not pd.isnull(c)])


def concat_set(df):
    _concat = lambda x: ', '.join(x) if isinstance(x, set) else x
    string_dtypes = df.select_dtypes(exclude=[np.number])
    df[string_dtypes.columns] = string_dtypes.applymap(_concat)
    return df

for taxon_group, postprocessor in postprocessors.items():
    excel_file = RAW_DATA_DIR / f'{taxon_group}.xlsx'
    
    df = postprocessor.todf()
    combined = df.groupby('taxon').aggregate(aggregate)
    # combined = combined.drop(columns=['source'])
    concat_set(df)
    concat_set(combined)

    with pd.ExcelWriter(excel_file) as writer:
        combined.to_excel(writer, sheet_name="Combined")
        for source in df.source.unique():        
            df[df.source == source].to_excel(writer, sheet_name=source, index=False)
        
#         print(taxon_group)
#         print(postprocessor._data)

In [53]:


def aggregate(x):
    try:
        if x.dtype in ['int64', 'float64']:
            return round(x.mean(), 2)
        else:
            return set.union(*list(filter(None, x)))
    except:
        return 'ERR'
    
def concat_set(x):
    return ', '.join(x)    
    
# df = postprocessor.todf()
combined = df.groupby('taxon').aggregate(aggregate) 

# string_dtypes = combined.select_dtypes(exclude=[np.number])
# combined[string_dtypes.columns] = string_dtypes.applymap(concat_set)

combined.head()

# print(string_dtypes)
# combined.select_dtypes(exclude=[np.number])

# .applymap(concat_set)

# combined.head()

# print(combined.select_dtypes(exclude=[np.number]))

# combined[combined.dtypes != "object"].applymap(lambda x: ', '.join(x))
    

    
# combined = df.groupby('taxon').aggregate(aggregate)

# import numpy as np
# combined.select_dtypes(exclude=[np.number]).applymap(lambda x: ', '.join(x))

# print(combined)
# # df.dtypes

,life form,habitat,clonality,plant min. height [m],plant max. height [m],indumentum,spinescence,succulence,leaf architecture,leaf position,...,dispersule max. length [cm],seeds per fruit_max.,seeds per fruit_min.,seed min. width [mm],seed max. width [mm],seed min. length [mm],seed max. length [mm],max seed volume,root min. depth [cm],root max. depth [cm]
taxon,,,,,,,,,,,,,,,,,,,,,
Achillea millefolium,{perennial},NaN,"{rhizomatous, solitary plant, stoloniferous}",0.14,0.56,"{woolly, pubescent, bristly, hairy, tomentose,...",NaN,NaN,"{lobed, sessile, pinnatipartite}",NaN,...,NaN,NaN,NaN,1.8,2.0,1.65,1.68,NaN,NaN,NaN


In [54]:
latest_columns = [
    "life form",
    "habitat",
    "habit",
    "clonality",
    "perennial organ",
    "plant min height [m]",
    "plant max height [m]",
    "indumentum",
    "spinescence",
    "succulence",
    "leaf arrangement",
    "leaf architecture",
    "leaf position",
    "leaf shape",
    "leaf apex",
    "leaf base",
    "leaf margin",
    "leaf defence",
    "leaf min width [cm]",
    "leaf max width [cm]",
    "leaf min length [cm]",
    "leaf max length [cm]",
    "inflorescence arrangement",
    "flower sex",
    "flower architecture",
    "flower merosity",
    "flower symmetry",
    "flower shape",
    "flower colour",
    "petal fusion",
    "petal colour",
    "reproduction architecture",
    "reproduction system",
    "stamen count",
    "stamen number",
    "stamen arrangement",
    "carpel/ovary number",
    "gynoecium arrangement",
    "heterostyly",
    "pollination",
    "fruit type",
    "fruit structure",
    "fruit dehiscence",
    "fruit shape",
    "fruit colour",
    "dispersule min width [cm]",
    "dispersule max width [cm]",
    "dispersule min length [cm]",
    "dispersule max length [cm]",
    "dispersal mode",
    "dispersion axillary",
    "seed colour",
    "seeds max per fruit",
    "seeds min per fruit",
    "seed min. width [mm]",
    "seed max. width [mm]",
    "seed min. length [mm]",
    "seed max. length [mm]",
    "max seed volume",
    "ploidy",
    "ploidy level",
    "root system",
    "root depth type",
    "root depth [cm]",
]

In [86]:
from adept.config import RAW_DATA_DIR
import pandas as pd

# columns = {}

# xls = pd.ExcelFile(RAW_DATA_DIR / 'functional-trait-list.xlsx')

# for sheet_name in xls.sheet_names:
#     df = pd.read_excel(RAW_DATA_DIR / 'functional-trait-list.xlsx', sheet_name=sheet_name)
#     taxon_group = sheet_name.replace('traits', '').strip().lower()
#     columns[taxon_group] = [col.lower() for col in df.columns.to_list()]
    
    
columns['angiosperm'] += ['root min. depth [cm]', 'root max. depth [cm]']

# Replace plant height with min/ax
idx = columns['angiosperm'].index('plant height [m]')
columns['angiosperm'][idx:idx+1] = ['plant min. height [m]', 'plant max. height [m]']


print(columns['angiosperm'])


['life form', 'habitat', 'habit', 'clonality', 'perennial organ', 'plant min. height [m]', 'plant max. height [m]', 'indumentum', 'spinescence', 'succulence', 'leaf arrangement', 'leaf architecture', 'leaf position', 'leaf shape', 'leaf apex', 'leaf base', 'leaf margin', 'leaf min. width [cm]', 'leaf max. width [cm]', 'leaf min. length [cm]', 'leaf max. length [cm]', 'inflorescence arrangement', 'flower sex', 'flower architecture', 'flower merosity', 'flower symmetry', 'flower shape', 'flower colour', 'petal fusion', 'petal colour', 'reproduction architecture', 'reproduction system', 'stamen number', 'stamen arrangement', 'carpel/ovary number', 'gynoecium arrangement', 'heterostyly', 'pollination', 'fruit type', 'fruit structure', 'fruit dehiscence', 'fruit shape', 'fruit colour', 'dispersule min. width [cm]', 'dispersule max. width [cm]', 'dispersule min. length [cm]', 'dispersule max. length [cm]', 'seed colour', 'seeds per fruit_max.', 'seeds per fruit_min.', 'seed min. width [mm]',

In [81]:
# print(columns['angiosperm'])

old_cols = [c.lower() for c in columns['angiosperm']]


# dimensions = {'length', 'width', 'depth', 'height'}

# for col in old_cols:
#     col_parts = set(col.split())
#     if col_parts.intersection(dimensions):
#         print(col_parts)





old_cols += ['root min. depth [cm]', 'root max. depth [cm]']

# Replace plant height with min/ax
idx = old_cols.index('plant height [m]')
old_cols[idx:idx+1] = ['plant min. height [m]', 'plant max. height [m]']

# index

old_cols

# root depth [cm]

['life form',
 'habitat',
 'habit',
 'clonality',
 'perennial organ',
 'plant min. height [m]',
 'plant max. height [m]',
 'indumentum',
 'spinescence',
 'succulence',
 'leaf arrangement',
 'leaf architecture',
 'leaf position',
 'leaf shape',
 'leaf apex',
 'leaf base',
 'leaf margin',
 'leaf min. width [cm]',
 'leaf max. width [cm]',
 'leaf min. length [cm]',
 'leaf max. length [cm]',
 'inflorescence arrangement',
 'flower sex',
 'flower architecture',
 'flower merosity',
 'flower symmetry',
 'flower shape',
 'flower colour',
 'petal fusion',
 'petal colour',
 'reproduction architecture',
 'reproduction system',
 'stamen number',
 'stamen arrangement',
 'carpel/ovary number',
 'gynoecium arrangement',
 'heterostyly',
 'pollination',
 'fruit type',
 'fruit structure',
 'fruit dehiscence',
 'fruit shape',
 'fruit colour',
 'dispersule min. width [cm]',
 'dispersule max. width [cm]',
 'dispersule min. length [cm]',
 'dispersule max. length [cm]',
 'seed colour',
 'seeds per fruit_max.',

In [46]:
x = [set([1]), set([2,3]), None]

In [52]:
# set.union(filter(None, *x))

set.union(*list(filter(None, x)))

{1, 2, 3}

In [83]:
df = pd.DataFrame([{'a': 1, 'b': 2}])

In [87]:
# df[['a']]['b'] = df[['a']]

In [88]:
missing = [{'term': 'stem erect', 'character': 'stem erect', 'trait': 'habit'}]

,a,b
0,1,2


In [148]:
from adept.traits.accdb import ACCDBTraits
accdb = ACCDBTraits()

df = accdb.get_plant_group('angiosperm')

df2 = df[['character', 'trait', 'term']]

# df2['term'] = df.character




In [149]:
df2[df2.character == 'stoloniferous']

,character,trait,term
2466,stoloniferous,clonality,stoloniferous
2991,stoloniferous,clonality,stolon
3768,stoloniferous,clonality,stolons
4730,stoloniferous,clonality,stolonoid


In [8]:
tt[tt.term == 'woolly']

,term,character,trait,Plants Group
599,woolly,hairs absent,indumentum,angiosperm
2842,woolly,woolly,indumentum,angiosperm


In [10]:
tt.shape

(2640, 4)

In [55]:
a = [{'a':1},{'b':2},{'c':1},{'d':2}]
b = [{'life cycle': {'perennial'}}, {'habit': {'erect leafy', 'scrambler', 'herb'}}, {'clonality': {'stoloniferous', 'stolon', 'stolons'}}, {'perennial organ': {'stem', 'stolon'}}, {'indumentum': {'woolly', 'hairs absent', 'glabrous'}}, {'leaf shape': {'lanceolate', 'setiform', 'linear'}}, {'leaf apex': {'lanceolate', 'subulate'}}, {'leaf arrangement': {'pinnate'}}, {'leaf architecture': {'pinnate', 'sessile'}}, {'inflorescence arrangement': {'capitulum', 'corymb'}}, {'flower structure': {'naked'}}, {'flower colour': {'brown', 'black', 'white'}}, {'dispersal mode': {'anemochory'}}, {'dispersion axillary': {'seed winged'}}]

In [56]:
dict(ChainMap(*b))

{'dispersion axillary': {'seed winged'},
 'dispersal mode': {'anemochory'},
 'flower colour': {'black', 'brown', 'white'},
 'flower structure': {'naked'},
 'inflorescence arrangement': {'capitulum', 'corymb'},
 'leaf architecture': {'pinnate', 'sessile'},
 'leaf arrangement': {'pinnate'},
 'leaf apex': {'lanceolate', 'subulate'},
 'leaf shape': {'lanceolate', 'linear', 'setiform'},
 'indumentum': {'glabrous', 'hairs absent', 'woolly'},
 'perennial organ': {'stem', 'stolon'},
 'clonality': {'stolon', 'stoloniferous', 'stolons'},
 'habit': {'erect leafy', 'herb', 'scrambler'},
 'life cycle': {'perennial'}}

In [13]:
xslx = RAW_DATA_DIR / 'functional-trait-list.xlsx'

In [14]:
df = pd.read_excel(xslx, sheet_name='Angiosperm traits')

In [15]:
df.head()

,Life form,Habitat,Habit,Clonality,Perennial Organ,Plant height [m],Indumentum,Spinescence,Succulence,LEAF Arrangement,...,SEEDS per fruit_max.,SEEDS per fruit_min.,Seed min. width [mm],Seed max. width [mm],Seed min. length [mm],Seed max. length [mm],Dispersal mode,Dispersion axillary,Ploidy level (2n),Max seed volume
0,annual,aquatic,climber,agamospermy,bulb,NaN,bristly,aculeate,yes,aphyllous,...,NaN,NaN,NaN,NaN,NaN,NaN,airborne,aril,NaN,NaN
1,biennial,epiphytic,cushion,bulb,corm,NaN,ciliate,armed,no,bifoliolate,...,NaN,NaN,NaN,NaN,NaN,NaN,anemochory,awn,NaN,NaN
2,evergreen,geophyte,erect leafy [herb],bulbils,rhizome,NaN,glabrescent,spiculose,NaN,bipinnate,...,NaN,NaN,NaN,NaN,NaN,NaN,ectzoochory,bristles,NaN,NaN
3,perennial,hemiparasite,mat,corm,stem,NaN,glabrous,spiny,NaN,digitate,...,NaN,NaN,NaN,NaN,NaN,NaN,hydrochory,filaments,NaN,NaN
4,NaN,hygrophilous,scrambler,gemmae,stipe,NaN,glandular,unarmed,NaN,pedate,...,NaN,NaN,NaN,NaN,NaN,NaN,mechanical,hairs,NaN,NaN


In [22]:
data = []

for col in df.columns:
    values = df[col].dropna().unique()
    if len(values) > 2:
        for value in values:
            data.append(
                {'term': value, 'character': value, 'trait': col.lower()})
            
print(data)

[{'term': 'annual', 'character': 'annual', 'trait': 'life form'}, {'term': 'biennial', 'character': 'biennial', 'trait': 'life form'}, {'term': 'evergreen', 'character': 'evergreen', 'trait': 'life form'}, {'term': 'perennial', 'character': 'perennial', 'trait': 'life form'}, {'term': 'aquatic', 'character': 'aquatic', 'trait': 'habitat'}, {'term': 'epiphytic', 'character': 'epiphytic', 'trait': 'habitat'}, {'term': 'geophyte', 'character': 'geophyte', 'trait': 'habitat'}, {'term': 'hemiparasite', 'character': 'hemiparasite', 'trait': 'habitat'}, {'term': 'hygrophilous', 'character': 'hygrophilous', 'trait': 'habitat'}, {'term': 'lithophytic', 'character': 'lithophytic', 'trait': 'habitat'}, {'term': 'parasite', 'character': 'parasite', 'trait': 'habitat'}, {'term': 'riparian', 'character': 'riparian', 'trait': 'habitat'}, {'term': 'saprophytic', 'character': 'saprophytic', 'trait': 'habitat'}, {'term': 'terrestrial', 'character': 'terrestrial', 'trait': 'habitat'}, {'term': 'climber',

In [24]:
from adept.traits.accdb import ACCDBTraits
accdb = ACCDBTraits()

In [27]:
df = accdb._df

df[df.trait == 'spinescence']

,termID,term,character,trait,Plants Group,synonym
155,46397558-1ce7-11e4-990a-0026b9326338,branching-spiculose,spiculose,spinescence,bryophyte,NaN
223,46584ece-1ce7-11e4-990a-0026b9326338,spinulescent,spinulate,spinescence,bryophyte,NaN
230,fcb53ec7-d7c8-42b9-9a18-4e5f41454581,spineless,unarmed,spinescence,angiosperm,NaN
244,d412e218-5039-4f84-8a20-f4895d511611,spine,spinescent,spinescence,pteridophyte,NaN
248,d412e218-5039-4f84-8a20-f4895d511611,spine,spinescent,spinescence,angiosperm,NaN
...,...,...,...,...,...,...
6929,NaN,aculeate,aculeate,spinescence,angiosperm,NaN
6930,NaN,armed,armed,spinescence,angiosperm,NaN
6931,NaN,spiculose,spiculose,spinescence,angiosperm,NaN
6932,NaN,spiny,spiny,spinescence,angiosperm,NaN
